In [14]:
import sys
sys.path.append('../')
import matlab.engine
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from skopt.sampler import Lhs
from skopt.space import Space
import math
from pyswarms.single.global_best import GlobalBestPSO
import pyswarms as ps
from scipy.stats import norm
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)
from sklearn.neighbors import KernelDensity
from numpy.random import uniform
from sklearn.preprocessing import MinMaxScaler
import matplotlib.transforms as transforms
from matplotlib.patches import Ellipse
from Hypervolume import HyperVolume
import optproblems.zdt
import ConfigSpace as CS
import sys
import matplotlib.pyplot as plt
from SVM import ML as ML_SVM
from DT import ML as ML_DT
from ML import ML as ML_MLP
from Problem import Problem
from MOTPE import MOTPE

eng = matlab.engine.start_matlab()
random_seed = 1990

In [15]:
def plot_pf(non_dominated_points, solutions):
    
    m = np.mean(non_dominated_points, axis=1)
    std =  np.sqrt(np.var(non_dominated_points, axis=1))
    max_data = m+std #np.max(non_dominated_points, axis=1)
    min_data = m-std #np.min(non_dominated_points, axis=1)
    
    ells = [Ellipse(xy=[m[i,0], m[i,1]], 
                    width=np.abs(max_data[i,0]-min_data[i,0]), 
                    height=np.abs(max_data[i,1]-min_data[i,1]))
            for i in range(m.shape[0])]

    fig, ax = plt.subplots(figsize=(10,10))

    index = 0
    for e in ells:
        ax.add_patch(e)
        
        if index==0:
            e.set(clip_box=ax.bbox, alpha=0.2, facecolor="peachpuff", edgecolor='silver', label='Uncertainty')
        else:
            e.set(clip_box=ax.bbox, alpha=0.2, facecolor="peachpuff", edgecolor='silver')
        index+=1
        
#         e.set_edgecolor('silver')
    max_axes = np.max([max_data[:,0], max_data[:,1]])
#     ax.set_xlim(0, max_axes+.5)
#     ax.set_ylim(0, max_axes+.5)

    ax.scatter(m[:, 0], m[:, 1], color='k', s=50, 
               zorder=len(ells)+1, label="Observed PF")
#     ax.scatter(tf[:,0], tf[:,1], label="Optimal PF")
    ax.scatter(solutions[:, 0], solutions[:, 1], color='orange', marker='+', zorder=len(ells)+2, label="Solutions")
    ax.legend()
    return ax

In [16]:
def compute_hypervolume(m, referencePoint):
    hv = HyperVolume(referencePoint)
    volume = hv.compute(m)
    return volume

In [17]:
def pareto_front(points, level=0, index=False):

    points = points.copy()
    ranks = np.zeros(len(points))
    r = 0
    c = len(points)
    while c > 0:
        extended = np.tile(points, (points.shape[0], 1, 1))
        dominance = np.sum(np.logical_and(
            np.all(extended <= np.swapaxes(extended, 0, 1), axis=2),
            np.any(extended < np.swapaxes(extended, 0, 1), axis=2)), axis=1)
        points[dominance == 0] = 1e9  # mark as used
        ranks[dominance == 0] = r
        r += 1
        c -= np.sum(dominance == 0)
    if index:
#         return ranks
        return [i for i in range(len(ranks)) if ranks[i] == level] 
    else:    
        ind = [i for i in range(len(ranks)) if ranks[i] == level]
        return points[ind]

In [18]:
def acquisition_function(candidates, model, Z_min, score_tpe=0):
    BK = np.asarray([np.ones(candidates.shape[0])])
    BK = matlab.double(np.transpose(BK).tolist())

    [SK_gau, SK_gau_mse, det_mse]  = eng.SKpredict_new(model, matlab.double(candidates.tolist()), BK, 
                                              matlab.double(np.asarray([random_seed]).tolist()),
                                              nargout=3)
    SK_gau = np.asarray(SK_gau)
    det_mse = np.asarray(det_mse)
    mse = np.sqrt(np.abs(det_mse))
    
    mei = (Z_min-SK_gau) * norm.cdf((Z_min-SK_gau)/mse,0,1) + mse * norm.pdf((Z_min-SK_gau)/mse,0,1) + score_tpe
#     print("mei", mei, mei.shape)
    return -mei[:,0]

In [19]:
eps = sys.float_info.epsilon


variables_array = [5, 2, 5]
ML_names = ["MLP", "SVM", "DT"]
datasets_id = [803, 980, 819, 871] #41146, 847, 


for id_d in datasets_id: # For each dataset
    for v_index in range(len(variables_array)):
        
        num_variables =variables_array[v_index] # Obtain the number of hyperparameters
        ML_algorithm = ML_names[v_index] # obtain the name of ML algorithm

        seed                = random_seed   #args.seed
        num_initial_samples = 11*num_variables-1    #args.num_initial_samples
        num_max_evals       = 100   #args.num_max_evals
        num_objectives      = 2     #args.num_objectives
        k                   = 1     #args.k
        num_candidates      = 1000    #args.num_candidates
        init_method         = 'random' #args.init_method
        gamma               = 0.1   #args.gamma
        replications        = 10
        # LOAD DATA

        dataset = fetch_openml(data_id=id_d)
        X = dataset.data
        y = dataset.target
        # X_train and y_train will be used in an internal cross validation process for the HPO
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=seed)

        base_configuration  = {
            'num_objectives': num_objectives,
            'num_variables': num_variables,
            'replications': replications,
            'k': k,
            'seed': seed}

        if ML_algorithm == "MLP":
            f = ML_MLP(ML_algorithm, X_train, y_train, X_test, y_test, base_configuration)
        elif ML_algorithm == "SVM":
            f = ML_SVM(ML_algorithm, X_train, y_train, X_test, y_test, base_configuration)
        else:
            f = ML_DT(ML_algorithm, X_train, y_train, X_test, y_test, base_configuration)

        cs      = f.make_cs(CS.ConfigurationSpace(seed=seed))
        problem = Problem(f, cs)
        solver  = MOTPE(seed=seed)

        cs, i = solver.init_model(
            {'num_initial_samples': num_initial_samples,
             'num_max_evals': num_max_evals,
             'init_method': init_method,
             'num_candidates': num_candidates,
             'scalarize':False,
             'gamma': gamma},
                problem)
        # data = solver.history_replications()
        # X = solver.get_X()
        hv_history = []
        W = solver.get_weights_scalarization()
        
        try:
            #Optimization procedure
            for it in range(num_max_evals):   
                print("Iteration: ", it, "/", num_max_evals)

                data = solver.history_replications()
                X = solver.get_X()

                #Prepare de optimization
                Y_mean=np.zeros(data.shape[0])
                Vhat=np.zeros(data.shape[0])
                B = np.asarray([np.ones(data.shape[0])])

                #### Scalarization
                # Randomly select a point
                weight = W[np.random.randint(0, W.shape[0]),:]

                for i in range(data.shape[0]):
                    point = np.asarray(data[i])
                    pcheby_term1 = np.max(point * np.tile(weight, (replications,1)), axis=1)
                    pcheby_term2 = np.sum(point * np.tile(weight, (replications,1)), axis=1)
                    pcheby = pcheby_term1 + 0.05*pcheby_term2
                    Y_mean[i] = np.mean(pcheby);
                    Vhat[i] = np.var(pcheby)/replications;
                Vhat = np.asarray([Vhat]).T
                #Normalize objectives
                scaler = MinMaxScaler()
                Y_mean = scaler.fit_transform(np.asarray([Y_mean]).T)


            #     print(X.shape, Y_mean.shape, B.T.shape, Vhat.shape)

                X_matlab = matlab.double(X.tolist())
                Y_matlab = matlab.double(Y_mean.tolist())
                B = matlab.double(np.transpose(B).tolist())
                Vhat = matlab.double(Vhat.tolist())

                skriging_model_2 = eng.SKfit_new(X_matlab,Y_matlab, B, Vhat, 2, 3, 
                                                 matlab.double(np.asarray([random_seed]).tolist()), nargout=1);

            #     ######### SELECTION ########
                # Select the best point and evaluate it wh SK
                index_best = Y_mean.argmin(axis=0)[0]
                X_matlab = matlab.double(np.asarray(X[index_best,:]).tolist())

                [SK_gau, SK_gau_mse]  = eng.SKpredict_new(skriging_model_2, X_matlab, 
                                                          matlab.double(np.asarray([1]).tolist()), 
                                                          matlab.double(np.asarray([random_seed]).tolist()),
                                                          nargout=2);
                SK_gau_array = np.asarray(SK_gau)
                sample = solver.sample(cs)


                # Select the point sampled from l that maximize the MEI
                meis = acquisition_function(sample, skriging_model_2, SK_gau_array)
                best_mei = meis.argmin(axis=0)
                pos = sample[best_mei]
            #     print(pos)
                solver.add_infill_point(problem, pos, i)

        # Save all the solutions found
            m = solver.get_F()
            df = pd.DataFrame(m)
            df.to_csv("PF_data/ML/New_datasets/GP_MOTPE/exploration_GP_MOTPE_val_"+ML_algorithm+"_"+str(id_d)+".csv")

            # Save the evaluation on test set for all the solutions found
            m = solver.test_evaluations()
            df = pd.DataFrame(m)
            df.to_csv("PF_data/ML/New_datasets/GP_MOTPE/exploration_GP_MOTPE_test_"+ML_algorithm+"_"+str(id_d)+".csv")

            rep = solver.history_replications()
            mmm = pd.DataFrame(rep.reshape(rep.shape[0]*rep.shape[1],2))
            mmm.to_csv("PF_data/ML/New_datasets/GP_MOTPE/replications_GP_MOTPE_"+ML_algorithm+"_"+str(id_d)+".csv")
        except:
            print("MATLAB execution error")
            print("Save ")
            m = solver.get_F()
            df = pd.DataFrame(m)
            df.to_csv("PF_data/ML/New_datasets/GP_MOTPE/exploration_GP_MOTPE_val_"+ML_algorithm+"_"+str(id_d)+".csv")

            # Save the evaluation on test set for all the solutions found
            m = solver.test_evaluations()
            df = pd.DataFrame(m)
            df.to_csv("PF_data/ML/New_datasets/GP_MOTPE/exploration_GP_MOTPE_test_"+ML_algorithm+"_"+str(id_d)+".csv")

            rep = solver.history_replications()
            mmm = pd.DataFrame(rep.reshape(rep.shape[0]*rep.shape[1],2))
            mmm.to_csv("PF_data/ML/New_datasets/GP_MOTPE/replications_GP_MOTPE_"+ML_algorithm+"_"+str(id_d)+".csv")

Sampling initial points: 
     0.0 %
     1.85 %
     3.7 %
     5.56 %
     7.41 %
     9.26 %
     11.11 %
     12.96 %
     14.81 %
     16.67 %
     18.52 %
     20.37 %
     22.22 %
     24.07 %
     25.93 %
     27.78 %
     29.63 %
     31.48 %
     33.33 %
     35.19 %
     37.04 %
     38.89 %
     40.74 %
     42.59 %
     44.44 %
     46.3 %
     48.15 %
     50.0 %
     51.85 %
     53.7 %
     55.56 %
     57.41 %
     59.26 %
     61.11 %
     62.96 %
     64.81 %
     66.67 %
     68.52 %
     70.37 %
     72.22 %
     74.07 %
     75.93 %
     77.78 %
     79.63 %
     81.48 %
     83.33 %
     85.19 %
     87.04 %
     88.89 %
     90.74 %
     92.59 %
     94.44 %
     96.3 %
     98.15 %
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100


C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  war

Iteration:  31 / 100
Iteration:  32 / 100
Iteration:  33 / 100
Iteration:  34 / 100
Iteration:  35 / 100
Iteration:  36 / 100
Iteration:  37 / 100
Iteration:  38 / 100
Iteration:  39 / 100
Iteration:  40 / 100
Iteration:  41 / 100
Iteration:  42 / 100
Iteration:  43 / 100
Iteration:  44 / 100
Iteration:  45 / 100
Iteration:  46 / 100
Iteration:  47 / 100
Iteration:  48 / 100
Iteration:  49 / 100
Iteration:  50 / 100
Iteration:  51 / 100
Iteration:  52 / 100
Iteration:  53 / 100
Iteration:  54 / 100
Iteration:  55 / 100
Iteration:  56 / 100
Iteration:  57 / 100
Iteration:  58 / 100
Iteration:  59 / 100
Iteration:  60 / 100


C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (94) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration:  61 / 100
Iteration:  62 / 100
Iteration:  63 / 100
Iteration:  64 / 100
Iteration:  65 / 100
Iteration:  66 / 100
Iteration:  67 / 100
Iteration:  68 / 100
Iteration:  69 / 100
Iteration:  70 / 100
Iteration:  71 / 100
Iteration:  72 / 100
Iteration:  73 / 100
Iteration:  74 / 100
Iteration:  75 / 100
Iteration:  76 / 100
Iteration:  77 / 100
Iteration:  78 / 100
Iteration:  79 / 100
Iteration:  80 / 100
Iteration:  81 / 100
Iteration:  82 / 100
Iteration:  83 / 100
Iteration:  84 / 100
Iteration:  85 / 100
Iteration:  86 / 100
Iteration:  87 / 100
Iteration:  88 / 100
Iteration:  89 / 100
Iteration:  90 / 100
Iteration:  91 / 100
Iteration:  92 / 100
Iteration:  93 / 100
Iteration:  94 / 100
Iteration:  95 / 100
Iteration:  96 / 100
Iteration:  97 / 100
Iteration:  98 / 100
Iteration:  99 / 100
Sampling initial points: 
     0.0 %
     4.76 %
     9.52 %
     14.29 %
     19.05 %
     23.81 %
     28.57 %
     33.33 %
     38.1 %
     42.86 %
     47.62 %
     52.38 %
    

...

Iteration:  79 / 100


C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (9) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (9) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (9) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (9) reached and the optimization hasn't converged yet.
  warning

Iteration:  80 / 100
Iteration:  81 / 100
Iteration:  82 / 100
Iteration:  83 / 100
Iteration:  84 / 100
Iteration:  85 / 100
Iteration:  86 / 100
Iteration:  87 / 100
Iteration:  88 / 100
Iteration:  89 / 100
Iteration:  90 / 100
Iteration:  91 / 100
Iteration:  92 / 100
Iteration:  93 / 100
Iteration:  94 / 100


C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (37) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\lucp10597\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (37) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration:  95 / 100
Iteration:  96 / 100
Iteration:  97 / 100
Iteration:  98 / 100
Iteration:  99 / 100
Sampling initial points: 
     0.0 %
     4.76 %
     9.52 %
     14.29 %
     19.05 %
     23.81 %
     28.57 %
     33.33 %
     38.1 %
     42.86 %
     47.62 %
     52.38 %
     57.14 %
     61.9 %
     66.67 %
     71.43 %
     76.19 %
     80.95 %
     85.71 %
     90.48 %
     95.24 %
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
Iteration:  16 / 100
Iteration:  17 / 100
Iteration:  18 / 100
Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
Iteration:  24 / 100
Iteration:  25 / 100
Iteration:  26 / 100
Iteration:  27 / 100
Iteration:  28 / 100
Ite